# Github repo

clone github repository

In [1]:
!git clone https://github.com/thomasmaliappis/context-group-detection.git

Cloning into 'context-group-detection'...
remote: Enumerating objects: 592, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 592 (delta 120), reused 182 (delta 80), pack-reused 367
Receiving objects: 100% (592/592), 1.48 MiB | 6.95 MiB/s, done.
Resolving deltas: 100% (304/304), done.


move in repository folder

In [2]:
%cd context-group-detection

/content/context-group-detection


change branch

In [3]:
!git checkout dante

Branch 'dante' set up to track remote branch 'dante' from 'origin'.
Switched to a new branch 'dante'


# Drive

mount drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Requirements
install missing packages

In [5]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 15.6 MB/s eta 0:00:00


In [6]:
!pip install pykalman

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pykalman: filename=pykalman-0.9.5-py3-none-any.whl size=48442 sha256=3c81cca521958df92e831bf6a440b8c964a88c7340bd6e2ae627369d018dd969
  Stored in directory: /root/.cache/pip/wheels/76/33/ef/5f332226e13a5089c6dd4b01cc2bcb59491d18f955fa2d3807
Successfully built pykalman


# Imports

In [7]:
import argparse
import os

import numpy as np
import tensorflow as tf
from keras.callbacks import EarlyStopping, TensorBoard
from keras.layers import Dense, Conv1D, LSTM, concatenate, Reshape, Dropout, BatchNormalization, Input, Flatten, MaxPooling1D, Conv2D
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
from sklearn.model_selection import train_test_split

from models.DANTE.utils import ValLoss, get_path, write_history

# Functions

create model

In [8]:
def conv(filters, reg, name=None):
    return Conv1D(filters=filters, kernel_size=1, padding='valid', kernel_initializer="he_normal",
                  use_bias='True', kernel_regularizer=reg, activation=tf.nn.relu, name=name)


def build_model(context_size, consecutive_frames, features, units, reg_amount, drop_amount, learning_rate):
    inputs = []

    # pair branch
    # create input layers
    pair_inputs = []
    for i in range(2):
        pair_input = Input(shape=(consecutive_frames, features), name='pair_{}'.format(i))
        pair_inputs.append(pair_input)
        inputs.append(pair_input)

    pair_layers = []
    for pair_input in pair_inputs:
        lstm = LSTM(64, return_sequences=True)(pair_input)
        pair_layers.append(lstm)

    pair_concatenated = concatenate(pair_layers)
    # pair_reshaped = Reshape((pair_concatenated.shape[1], 1))(pair_concatenated)
    pair_conv = Conv1D(filters=32, kernel_size=1, activation='relu', name='pair_conv')(pair_concatenated)
    # drop = Dropout(drop_amount)(pair_conv)
    # batch_norm = BatchNormalization()(drop)
    # max_pool = MaxPooling1D()(batch_norm)
    # drop = Dropout(drop_amount)(max_pool)
    # batch_norm = BatchNormalization()(drop)
    pair_layer = pair_conv

    # context branch
    context_inputs = []
    for i in range(context_size):
        context_input = Input(shape=(consecutive_frames, features), name='context_{}'.format(i))
        context_inputs.append(context_input)
        inputs.append(context_input)

    context_layers = []
    for context_input in context_inputs:
        lstm = LSTM(64, return_sequences=True)(context_input)
        context_layers.append(lstm)

    context_concatenated = concatenate(context_layers)
    # context_reshaped = Reshape((context_concatenated.shape[1], 1))(context_concatenated)
    context_conv = Conv1D(filters=32, kernel_size=3, activation='relu', name='context_conv')(context_concatenated)
    # drop = Dropout(drop_amount)(context_conv)
    # batch_norm = BatchNormalization()(drop)
    # max_pool = MaxPooling1D()(batch_norm)
    # drop = Dropout(drop_amount)(max_pool)
    # batch_norm = BatchNormalization()(drop)
    context_layer = context_conv

    # Concatenate the outputs of the two branches
    combined = concatenate([pair_layer, context_layer], axis=1)
    flatten = Flatten()(combined)
    combined_dense = Dense(64)(flatten)
    # Output layer
    output = Dense(1, activation='sigmoid')(combined_dense)

    # Create the model with two inputs and one output
    model = Model(inputs=[inputs], outputs=output)

    # Compile the model
    opt = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, decay=1e-5, amsgrad=False, clipvalue=0.5)
    model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['mse'])

    return model

split dataset into train, test and val sets

In [9]:
def train_test_split_frames(frames):
    """
    Split train, test and val indices.
    :param frames: list of frames
    :return: train, test and val indices
    """
    frame_ids = [frame[0] for frame in frames]
    frame_values = [list(x) for x in set(tuple(frame_id) for frame_id in frame_ids)]
    train, test = train_test_split(frame_values, test_size=0.3, random_state=0)
    idx_train = [i for i, frame in enumerate(frame_ids) if frame in train]
    frame_ids_train = [frame[0] for frame in frames[idx_train]]
    frame_values_train = [list(x) for x in set(tuple(frame_id) for frame_id in frame_ids_train)]
    train, val = train_test_split(frame_values_train, test_size=0.2, random_state=0)
    idx_train = [i for i, frame in enumerate(frame_ids) if frame in train]
    idx_test = [i for i, frame in enumerate(frame_ids) if frame in test]
    idx_val = [i for i, frame in enumerate(frame_ids) if frame in val]
    return idx_train, idx_test, idx_val


def train_test_split_groups(groups, frames_train, frames_test, frames_val):
    """
    Split groups in train, test and val groups.
    :param groups: list of groups per frame
    :param frames_train: list of train frames
    :param frames_test: list of test frames
    :param frames_val: list of val frames
    :return:
    """
    frame_ids_train = [frame[0] for frame in frames_train]
    frame_values_train = [list(x) for x in set(tuple(frame_id) for frame_id in frame_ids_train)]
    frame_ids_test = [frame[0] for frame in frames_test]
    frame_values_test = [list(x) for x in set(tuple(frame_id) for frame_id in frame_ids_test)]
    frame_ids_val = [frame[0] for frame in frames_val]
    frame_values_val = [list(x) for x in set(tuple(frame_id) for frame_id in frame_ids_val)]
    groups_train = [group for group in groups if group[0] in frame_values_train]
    groups_test = [group for group in groups if group[0] in frame_values_test]
    groups_val = [group for group in groups if group[0] in frame_values_val]
    return groups_train, groups_test, groups_val


def load_data(path, agents):
    """
    Load dataset and reformat it to match model input.
    :param path: string of path to data
    :param agents: number of agents
    :return: train, test and val data
    """
    X = np.load(path + '_data.npy')
    y = np.load(path + '_labels.npy')
    frames = np.load(path + '_frames.npy', allow_pickle=True)
    groups = np.load(path + '_groups.npy', allow_pickle=True)

    samples = 0
    for frame in frames:
        if frame[0] == frames[0][0] and frame[1] == frames[0][1]:
            samples += 1

    idx_train, idx_test, idx_val = train_test_split_frames(frames)
    groups_train, groups_test, groups_val = \
        train_test_split_groups(groups, frames[idx_train], frames[idx_test], frames[idx_val])

    train = ([X[idx_train, :, i] for i in range(agents)], y[idx_train], frames[idx_train], groups_train)
    test = ([X[idx_test, :, i] for i in range(agents)], y[idx_test], frames[idx_test], groups_test)
    val = ([X[idx_val, :, i] for i in range(agents)], y[idx_val], frames[idx_val], groups_val)
    return train, test, val, samples

class to handle arguments instead of argument parser

In [10]:
class Args:
    def __init__(self, dataset, dataset_path, epochs, features, agents, frames, batch_size, reg, dropout, learning_rate):
        self.dataset = dataset
        self.dataset_path = dataset_path
        self.epochs = epochs
        self.features = features
        self.agents = agents
        self.frames = frames
        self.batch_size = batch_size
        self.reg = reg
        self.dropout = dropout
        self.learning_rate = learning_rate


# Main

set argument constants

In [11]:
args = Args(
    dataset='eth',
    dataset_path='./datasets/ETH/seq_eth',
    epochs=40,
    features=4,
    agents=10,
    frames=10,
    batch_size=1024,
    reg=0.0000001,
    dropout=0.35,
    learning_rate=0.0001
)

load data

In [12]:
train, test, val, samples = load_data(
    '/content/drive/My Drive/datasets/{}_{}_{}'.format(args.dataset, args.frames, args.agents), args.agents)

X_train, y_train, frames_train, groups_train = train
X_val, y_val, frames_val, groups_val = val

initialise variables

In [13]:
tf.random.set_seed(0)
np.random.seed(0)
best_val_mses = []
best_val_f1s_one = []
best_val_f1s_two_thirds = []

create and train model

In [14]:
model = build_model(args.agents - 2, args.frames, args.features, 64, args.reg, args.dropout, args.learning_rate)

early_stop = EarlyStopping(monitor='val_loss', patience=10)
history = ValLoss(val, args.dataset, args.dataset_path, samples, True)
tensorboard = TensorBoard(log_dir='./logs')

model.fit(X_train, y_train,
          epochs=args.epochs, batch_size=args.batch_size,
          validation_data=(X_val, y_val),
          callbacks=[early_stop, history, tensorboard]
          )

Epoch 1/40
 5/42 [==>...........................] - ETA: 1s - loss: 0.6842 - mse: 0.2456

42/42 [==============================] - 39s 382ms/step - loss: 0.6660 - mse: 0.2367 - val_loss: 0.6586 - val_mse: 0.2330
Epoch 2/40
42/42 [==============================] - 7s 164ms/step - loss: 0.6186 - mse: 0.2139 - val_loss: 0.5917 - val_mse: 0.2014
Epoch 3/40
42/42 [==============================] - 8s 193ms/step - loss: 0.5565 - mse: 0.1853 - val_loss: 0.5032 - val_mse: 0.1617
Epoch 4/40
42/42 [==============================] - 6s 154ms/step - loss: 0.4806 - mse: 0.1529 - val_loss: 0.4117 - val_mse: 0.1249
Epoch 5/40
42/42 [==============================] - 8s 192ms/step - loss: 0.4072 - mse: 0.1240 - val_loss: 0.3455 - val_mse: 0.1029
Epoch 6/40
42/42 [==============================] - 8s 186ms/step - loss: 0.3447 - mse: 0.1010 - val_loss: 0.2919 - val_mse: 0.0849
Epoch 7/40
42/42 [==============================] - 8s 192ms/step - loss: 0.2977 - mse: 0.0851 - val_loss: 0.2569 - val_mse: 0.0746
Epoch 8/40
42/42 [==============================] - 8s 203ms/step - loss: 0.2608 - mse

keep track of evaluation metrics

In [15]:
best_val_mses.append(history.best_val_mse)
best_val_f1s_one.append(history.val_f1_one_obj['best_f1'])
best_val_f1s_two_thirds.append(history.val_f1_two_thirds_obj['best_f1'])

# save model
path = get_path(args.dataset, False)
file = open(path + '/architecture.txt', 'w+')
file.write(
    # "global: " + str(global_filters) + "\nindividual: " +
    # str(individual_filters) + "\ncombined: " + str(combined_filters) +
    "\nreg= " + str(args.reg) + "\ndropout= " + str(args.dropout))
name = path + '/val_fold_' + str(0)
if not os.path.isdir(name):
    os.makedirs(name)

write_history(name + '/results.txt', history, test, samples, True)

history.val_f1_one_obj['model'].save(name + '/best_val_model.h5')
print("saved best val model as " + '/best_val_model.h5')

file.write("\n\nbest overall val loss: " + str(min(best_val_mses)))
file.write("\nbest val losses per fold: " + str(best_val_mses))

file.write("\n\nbest overall f1 1: " + str(max(best_val_f1s_one)))
file.write("\nbest f1 1s per fold: " + str(best_val_f1s_one))

file.write("\n\nbest overall f1 2/3: " + str(max(best_val_f1s_two_thirds)))
file.write("\nbest f1 2/3s per fold: " + str(best_val_f1s_two_thirds))

file.close()

saving model to models/eth/pair_predictions_1
891/891 [==============================] - 8s 9ms/step
saved best val model as /best_val_model.h5
